## 라이브러리 불러오기

In [2]:
# API 호출
# ## https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057440
from urllib.request import Request, urlopen
# Request https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.Request
# urlopen https://docs.python.org/ko/3/library/urllib.request.html#urllib.request.urlopen
from urllib.parse import urlencode, quote_plus
# urlenocde https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.urlencode
# quote_plus https://docs.python.org/ko/3/library/urllib.parse.html#urllib.parse.quote_plus
from urllib.parse import unquote
import requests

# 시간 생성
import time
import datetime
from pytz import timezone, utc

# 폴더 자동 생성
import os

# 코드 스케줄링
import threading
# import schedule

# XML 파싱
import xml.etree.ElementTree as ET

# 입력데이터 데이터프레임 작성
import pandas as pd
import numpy as np

# DataFrame 화면 출력                                                # DataFrame 화면 출력 설정이 변경되어 있습니다 #
from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows = None # default = 60

In [3]:
# 폴더 자동 생성 함수
# https://data-make.tistory.com/170
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

## 입력데이터

In [4]:
def getOnlyDirectory(index):
    
    global subCategoryDict, category
    rootDirectory = os.getcwd()
    
    subCategory = subCategoryDict[index]
    directory = rootDirectory + category + subCategory
    return directory

In [5]:
def getRouteIdList():
    folderPath = getOnlyDirectory('rid')
    fileName = 'routeIdList.txt'
    filePath = folderPath + fileName

    routeIdList = []
    if(os.path.isfile(filePath)):
        routeIdFile = open(filePath, 'r', encoding = 'utf-8', newline = '\n')
        # print('파일이 존재합니다.')

    else:
        createFolder(folderPath)
        routeIdFile = open(filePath, 'r', encoding = 'utf-8', newline = '\n')

    routeId = routeIdFile.readline()
    while(routeId):
        routeIdList.append(int(routeId))
        routeId = (routeIdFile.readline())

    routeIdFile.close()
    
    return routeIdList

## 출력함수

In [9]:
def getDirectory(index, routeId):
    global subCategoryDict, category
    rootDirectory = os.getcwd()
    # category = '/TEST'
    subCategory = subCategoryDict[index]
    directory = rootDirectory + category + subCategory + str(routeId) + '/'
    return directory

In [90]:
def makeStaFile(routeId):
    directory = getOnlyDirectory('station')
    createFolder(directory)
    stationFilePath = directory + str(routeId) + '.txt'
    print(stationFilePath)
    stationFile = open(stationFilePath, "w")
    
    return stationFile

In [79]:
subCategoryDict = {'dtt' : '/daily-time-table/', 
                   'dtt-log' : '/log/daily-time-table/', 
                   'rid' : '/route-id-list/',
                   'main-log' : '/log/main/',
                   'info' : '/BusInfo/',
                   'station' : '/station/',
                  }

In [7]:
isTest = int(input("테스트 버전 입니까?... True[1] OR False[0] : ")) 

##############################
##############################
                          ####
if(isTest):               ####
    category = '/TEST'    ####
                          ####
else:                     ####
    category = '/DATA'    ####
                          ####
##############################
##############################

테스트 버전 입니까?... True[1] OR False[0] :  1


In [93]:
routeIdList = []
routeIdList = getRouteIdList()

## KEY unquote

In [11]:
KEY = 'yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D' # 종현
# apiKEY = requests.utils.unquote(KEY)
apiKey = unquote(KEY)

## URL 생성

In [12]:
# 노선번호목록조회 busrouteservice
# 해당 노선번호의 노선ID, 노선유형, 운행지역 등을 제공합니다.
url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station' 
queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : '204000046' })
print(url+queryParams)

http://openapi.gbis.go.kr/ws/rest/busrouteservice/station?serviceKey=yEaR%2F3MDedRSlVJL%2F2pxnVg0yre1N5VF3RZ%2FUAt56MJ7J2mNpfqhUvy05pXV0uhHTVY7DbyCR8xmMaDdYga67Q%3D%3D&routeId=204000046


## API 호출

In [95]:
for routeId in routeIdList:
    
    staFile = makeStaFile(routeId)
    
    # 노선정보항목조회 busrouteservice/info
    # 해당 노선에 대한 노선번호, 기점/종점 정류소, 첫차/막차시간, 배차간격, 운행업체 등의 운행계획 정보를 제공합니다.
    url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/info'
    queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })
    
    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    busInfoXML = urlopen(request).read().decode('utf8')
    

    busInfoTree = ET.fromstring(busInfoXML)
    busRouteInfoItem = busInfoTree[2]
    
    for item in busRouteInfoItem:
        
        staFile.write(f"{item.find('routeName').text} ")            
        staFile.write(f"{item.find('routeId').text} ")         

        peekAlloc = item.find('peekAlloc').text

        timeString = item.find('upFirstTime').text
        upHour = int(timeString[:2])
        upMin = int(timeString[-2:])
        upTime = upHour * 60 + upMin

        timeString = item.find('upLastTime').text
        downHour = int(timeString[:2])
        downMin = int(timeString[-2:])
        downTime = downHour * 60 + downMin

        if(upTime > downTime):
            downTime += 24 * 60
        try:
            expTotal = str(int((downTime - upTime) / int(peekAlloc)) + 1)
        except:
            expTotal = 100
        
        staFile.write(f"{expTotal} ")
        
    # 총 정류장수를 첫줄 제일 마지막에 출력하고
    # 두번째 줄부터 정류장 수 만큼 정류장 이름(stationName)을 출력합니다.
    url = 'http://openapi.gbis.go.kr/ws/rest/busrouteservice/station'
    queryParams = '?' + urlencode({ quote_plus('serviceKey') : apiKey, quote_plus('routeId') : routeId })

    request = Request(url + queryParams)
    request.get_method = lambda: 'GET'
    busStationXML = urlopen(request).read().decode('utf8')

    busStationTree = ET.fromstring(busStationXML)
    busRouteStationItem = busStationTree[2]
    totalStation = len(busRouteStationItem)

    staFile.write(f"{totalStation}\n")

    for station in busRouteStationItem:
        
        staFile.write(f"{station.find('stationName').text}\n")
        
    staFile.close()

/home/ubuntu/Data-Structure-Project/TEST/station/200000104.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000108.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000109.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000119.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000120.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000145.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000175.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000193.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000193.txt
/home/ubuntu/Data-Structure-Project/TEST/station/200000205.txt
/home/ubuntu/Data-Structure-Project/TEST/station/204000046.txt
/home/ubuntu/Data-Structure-Project/TEST/station/204000056.txt
/home/ubuntu/Data-Structure-Project/TEST/station/204000057.txt
/home/ubuntu/Data-Structure-Project/TEST/station/204000059.txt
/home/ubuntu/Data-Structure-Project/TEST/station/204000065.txt
/home/ubuntu/Data-Structure-Project/TEST/station/204000